In [2]:
import pandas as pd
import pymysql
import sqlite3
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 150)

In [5]:
connection = sqlite3.connect("/Users/danielgilberg/data_science/soccer_matches_project/database.sqlite")

In [6]:
c = connection.cursor()

In [7]:
match_query = '''SELECT * FROM match'''
match_df = pd.read_sql(match_query, connection)

In [8]:
country_query = '''SELECT * FROM country'''
country_df = pd.read_sql(country_query, connection)

In [9]:
league_query = '''SELECT * FROM league'''
league_df = pd.read_sql(league_query, connection)

In [10]:
team_query = '''SELECT * FROM team'''
team_df = pd.read_sql(team_query, connection)

In [11]:
player_query = '''SELECT * FROM player'''
player_df = pd.read_sql(player_query, connection)

In [12]:
player_atts_query = '''SELECT * FROM Player_Attributes'''
player_atts_df = pd.read_sql(player_atts_query, connection)

In [14]:
team_atts_query = '''SELECT * FROM team_attributes'''
team_atts_df = pd.read_sql(team_atts_query, connection)

In [191]:
class soccerData(object):
    def __init__(self, match_query, country_df, team_df, player_df, team_atts_df, player_atts_df, league_df):
        self.match_df = match_df
        self.country_df = country_df
        self.team_df = team_df
        self.player_df = player_df
        self.team_atts_df = team_atts_df
        self.player_atts_df = player_atts_df
        self.league_df = league_df
        
    def get_scores(self):
        home_team = temp.match_df[["id", "home_team_api_id", "home_team_goal"]]
        away_team = temp.match_df[["id", "away_team_api_id", "away_team_goal"]]
        teams = temp.team_df[["team_api_id", "team_long_name"]]
        home_df = pd.merge(home_team, teams,left_on="home_team_api_id", right_on="team_api_id")
        home_df.rename(columns={"team_long_name": "home_team"}, inplace=True)
        home_df.drop(["team_api_id", "home_team_api_id"], axis=1, inplace=True)
        away_df = pd.merge(away_team, teams,left_on="away_team_api_id", right_on="team_api_id")
        away_df.rename(columns={"team_long_name": "away_team"}, inplace=True)
        away_df.drop(["away_team_api_id", "team_api_id"], axis=1, inplace=True)
        df = pd.merge(home_df, away_df,on="id")
        df["Winner"] = df.apply(lambda row: self.get_winner(row["home_team_goal"],
                                                           row["home_team"], row["away_team_goal"],
                                                           row["away_team"]), axis=1)
        df["Result"] = df.apply(lambda row: self.get_home_result(row["home_team_goal"],
                                                           row["home_team"], row["away_team_goal"],
                                                           row["away_team"]), axis=1)
        other_info = self.match_df[["id", "country_id", "league_id", "season", "date"]]
        df = pd.merge(df, other_info, on="id")
        return df
    

    def get_winner(self, home_score, home_team, away_score, away_team):
        if home_score > away_score:
            return home_team
        elif away_score > home_score:
            return away_team
        else:
            return "Draw"
        
    def get_home_result(self, home_score, home_team, away_score, away_team):
        if home_score > away_score:
            return "Home"
        elif away_score > home_score:
            return "Away"
        else:
            return "Draw"
        
    def get_home_win_pct_by_season(self):
        df = self.get_scores()
        test = df[df.Result == "Home"]
        home_win_count = test.groupby(["country_id", "season"], as_index=False).agg({"date": len})
        home_win_count.rename(columns={"date": "home_wins"}, inplace=True)
        game_count = df.groupby(["country_id", "season"], as_index=False).agg({"date": len})
        game_count.rename(columns={"date": "total_games"}, inplace=True)
        final = pd.merge(game_count, home_win_count, on=["country_id", "season"])
        final["home_win_pct"] = final.apply(lambda row: round(row["home_wins"]/row["total_games"], 3), axis=1)
        final = self.country_df.merge(final, left_on="id", right_on="country_id")
        final.drop(["id", "country_id"], axis=1, inplace=True)
        return final
    
    def get_win_pct_by_team(self):
        df = self.get_scores()
        teams = self.team_df.team_long_name.unique()
        rows = list()
        for team in teams:
            mask = (df.home_team == team) | (df.away_team==team)
            games = len(df[mask])
            wins = len(df[df.Winner == team])
            pct = round(wins/games, 3)
            row = [team, pct]
            rows.append(row)

        df = pd.DataFrame(rows, columns=["Team", "Win_Pct"])
        return df
    
    def get_win_pct_by_team_and_season(self):
        df = self.get_scores()
        teams = self.team_df.team_long_name.unique()
        rows = list()
        for team in teams:
            mask = (df.home_team == team) | (df.away_team==team)
            test = df[mask]
            seasons = test.season.unique()
            for season in seasons: 
                temp_df = test[test.season == season]
                games = len(temp_df)
                wins = len(temp_df[temp_df.Winner == team])
                pct = round(wins/games, 3)
                row = [team, season, pct]
                rows.append(row)
        final = pd.DataFrame(rows, columns= ["Team", "Season", "Win_Pct"])
        return final
        
    


temp = soccerData(match_df, country_df, team_df, player_df, team_atts_df, player_atts_df, league_df)

In [193]:
df = temp.get_win_pct_by_team_and_season()

In [196]:
df.sort_values("Win_Pct", ascending=False)

,Team,Season,Win_Pct
1012,FC Porto,2010/2011,0.900
642,Juventus,2013/2014,0.868
811,PSV,2014/2015,0.853
1088,SL Benfica,2015/2016,0.853
441,FC Bayern Munich,2012/2013,0.853
442,FC Bayern Munich,2013/2014,0.853
1273,Real Madrid CF,2011/2012,0.842
1283,FC Barcelona,2012/2013,0.842
444,FC Bayern Munich,2015/2016,0.824
1271,Real Madrid CF,2009/2010,0.816
